# Setup
Update path, load assemblies and import modules

In [18]:
import os
import sys
sys.path.extend([
    '.venv/Lib',
    '.venv/Lib/site-packages',
    '../LibSLH/bin/Release/netstandard2.0'])
import clr
sys.path.append('')

clr.AddReference('System.Drawing')
clr.AddReference('LibreMetaverse')
clr.AddReference('LibSLH')

# Commonly used
from OpenMetaverse import *
from LibSLH import *

# Important imports
from slhstudio import *
from helpers import *

import System.Drawing.Imaging
import math
import asyncio
from IPython.display import display, display_html, clear_output
from ipywidgets import widgets
from collections import defaultdict
from types import SimpleNamespace


# Start the Client
Can also be used to restart a new client instance

In [19]:
client = Client()
client.start()

client_namespace = SimpleNamespace()
def client_event_handler(event):
    return event_handler(client_namespace, event)

# Periodic update override
Make the client rotate in place by overriding the main update function. This ensures that world objects will begin to load for the client.

In [20]:
update_coros = {}
def spin_client(client):
    while True:
        for i in range(16):
            theta = math.pi * i / 16
            direction = Vector3(math.cos(theta), math.sin(theta), 0.0)
            # Oof gamer https://github.com/pythonnet/pythonnet/issues/906
            rotation = Vector3.RotationBetween(Vector3.UnitX, Vector3.Normalize(direction))
            target = Vector3.Add(client.Self.SimPosition, direction)  
            client.Self.Movement.Camera.LookAt(client.Self.SimPosition, target);
            client.Self.Movement.SendUpdate()
            yield True

update_coros['spin'] = spin_client(client)
async def my_update():
    for key in update_coros:
        next(update_coros[key])
    await asyncio.sleep(0.1)

client.update = my_update

# Examples

### Say a message in local chat

In [7]:
def say(obj):
    client.Self.Chat(str(obj), 0, ChatType.Normal)

In [9]:
say([x for x in range(10)])

### Misc. examples

In [12]:
avatars = {str(e.Key): str(e.Value) for e in client.Network.CurrentSim.AvatarPositions.Copy()}
say(avatars)
print(avatars)

{'b1d22e67-c078-4e34-9818-3f849b005686': '<124, 133, 568>', '4f8b7e1d-6ba4-42db-9fcc-6c21de52e8e1': '<127, 109, 996>', '4017a11f-b314-450f-9d5b-2f8a04bdc7b3': '<127, 113, 996>', '8aec473c-a544-482d-bc81-b8883ef10664': '<119, 204, 1020>', '142194dc-7775-4553-ae8a-d3b9910e792c': '<119, 116, 996>', '68dcb627-4953-4380-a2bf-90eb789e1ea6': '<97, 188, 24>', '5fe7946d-19ce-4543-bde8-3cbae8f93cfc': '<126, 111, 996>', '36481ba4-856a-49f1-8b05-a5921c7076bf': '<127, 109, 996>', 'de17eabb-ad31-47d1-9df2-799457824ceb': '<126, 113, 996>', '045edb89-8a00-40f6-bd9a-cd4ff4e386f7': '<126, 110, 996>', '4edadd12-606a-42de-be2f-06ce3a85853c': '<134, 109, 32>', 'de698c09-fd92-4e74-820d-3faad044659e': '<126, 110, 996>', '199dae5a-96c5-4400-91d9-c8dfdf91dd3a': '<247, 12, 1020>'}


In [10]:
{str(e.Key): str(e.Value) for e in client.Friends.FriendList.Copy()}

{'de17eabb-ad31-47d1-9df2-799457824ceb': 'Tarocco Banana (Their Rights: CanSeeOnline, My Rights: CanSeeOnline)'}

In [14]:
def debug_point_at(sender, args):
    pt = str(args.PointType)
    id = str(args.TargetID)
    pos = str(args.TargetPosition)
    message = f'{pt}, {id}, {pos}'
    client.Self.Chat(message, 0, ChatType.Normal)
remove_all_event_handlers(client.Avatars, 'ViewerEffectPointAt')
client.Avatars.ViewerEffectPointAt += debug_point_at

In [28]:
#dir(client.Inventory.BulkUpdateInventoryHandler)

### Accept teleport requests

In [21]:
# TODO
authorized_avatars = {}
authorized_av_tf = widgets.Text()
display(authorized_av_tf)

def authorized_av_tf_submit(sender):
    avatars = sender.value.split(',')
    query_ids = [UUID.Random() for _ in avatars]
    lookup = dict(zip(query_ids, avatars))
    def handle_reply(sender, args):
        # args.Avatars seems to always have just 1 element
        if args.QueryID in lookup:
            info = list(args.Avatars)[0]
            uuid, name = info.Key, info.Value
            authorized_avatars[uuid] = name
            del lookup[args.QueryID]
        if not lookup:
            print('done')
            client.Avatars.AvatarPickerReply -= handle_reply
        
    
    remove_all_event_handlers(client.Avatars, 'AvatarPickerReply')
    client.Avatars.AvatarPickerReply += handle_reply
    for qid in lookup:
        client.Avatars.RequestAvatarNameSearch(lookup[qid], qid)

@client_event_handler(client.Self.IM)
def handle_instant_message(sender, args):
    dialog = args.IM.Dialog
    if dialog == 22: # RequestTeleport
        from_agent_id = args.IM.FromAgentID
        if from_agent_id in authorized_avatars:
            session_id = args.IM.IMSessionID
            client.Self.TeleportLureRespond(from_agent_id, session_id, True)
    

authorized_av_tf.on_submit(authorized_av_tf_submit)

Text(value='')

done


### Debug Textures!

In [16]:
def get_prim_texture_info(prim):
    yield prim.Textures.DefaultTexture
    yield from filter(lambda t: t, prim.Textures.FaceTextures)

def display_multi_image(paths, max_width=256):
    tags = [f'<img src="{path}" style="display: inline-block; max-width: {max_width}px">' for path in paths]
    display_html('\n'.join(tags), raw=True)

def get_image_path(uuid):
    path = os.path.join('img_cache', str(uuid) + '.png')
    if not os.path.exists(path):
        client.GetTextureByUUID(uuid).Save(path)
    return path

@client_event_handler(client.Avatars.ViewerEffectPointAt)
def handle_viewer_effect_point_at(sender, args):
    if args.PointType == PointAtType.Select:
        local_id = client.GetPrimLocalId(args.TargetID)
        parent_id = client.GetParentLocalId(local_id)
        parent_id = parent_id or local_id
        link_set = list(client.GetLinkSetLocalIds(parent_id))
        prims = [client.Objects.GetPrimitive(args.Simulator, System.UInt32(id), UUID.Zero, False) for id in link_set]
        tex_info = [info for prim in prims for info in get_prim_texture_info(prim)]
        tex_uuids = list({info.TextureID for info in tex_info})
        clear_output()
        print("Loading images, please wait...")
        file_paths = [get_image_path(uuid) for uuid in tex_uuids]
        clear_output()
        display_multi_image(file_paths, 64)

Loading images, please wait...


In [ ]:
bar = 0
def foo(s, a):
    global bar
    print(bar)
    bar += 1

In [ ]:
event_handler

# Stop the client

In [29]:
client.stop()